In [1]:
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp

In [2]:
%matplotlib qt

In [ ]:
class Mols(Enum):
    dmag = "dmag"
    ocm = "ocm"
    butanol = "butanol"
    peta = "peta"


class W(Enum):
    x = "x"
    mass = "w_mass"
    vol = "w_vol"

In [ ]:
D_mean = 15000
DF=pd.read_csv('Datas/Viscosity_all_data.csv')

In [ ]:
# D-T plots
def plot_DT(compound: Mols, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (
        df["compound"] == Mols.butanol.value
    )

    data = df[compound_filter]
    grouped = data.groupby(by=[w.value])
    colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

    fig, ax = plt.subplots(num="DT")
    for _, group in grouped:
        x = group[w.value].mean()
        T = group["Temperature"]
        color = colors_list.pop()

        D = group["D"] / D_mean
        E = group["E"].mean()
        D0 = group["D0"].mean() / D_mean
        D_ols = D0 * np.exp(-E / (8.314 * (T + 273.15)))
        ax.plot(T, D_ols, label=f"{x: 5.2f} %", c=color)

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - T ({w.value})")
        ax.set_xlabel("Temperature [C]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


# D-x plots
def plot_Dx(compound: Mols, w: W):
    df = DF.copy(deep=True)
    compound_filter = (df["compound"] == compound.value) | (
        df["compound"] == Mols.butanol.value
    )
    data = df[compound_filter]
    temper_list = np.arange(15, 41, 5)
    colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

    fig, ax = plt.subplots(num="DX")
    for T in temper_list:
        cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
        group = data[cond]
        group = group[[w.value, "D"]].groupby(by=w.value).mean().reset_index()

        x = group[w.value]
        color = colors_list.pop()

        D = group["D"] / D_mean
        ax.plot(x, D, c=color, label=f"{T: 3.0f} C")

        ax.set_title(f"Butanol-1, {compound.value.upper()}: Diffusion - {w.value} (T)")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("D/D0")
    plt.legend()
    plt.show()


# E-x plots
def plot_Ex(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots(num="E")
    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["E", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["E"] / 1000
        ax.plot(
            [x.iloc[0], 100],
            [E.iloc[0], 19.660651841],
            # label=f"{name}",
        )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"E - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("E [kJ]")
    plt.legend()
    plt.show()


def plot_rho(w: W):
    df = DF.copy(deep=True)
    grouped = df.groupby(by=["compound"])
    fig, ax = plt.subplots()

    for _, group in grouped:
        name = group["compound"].unique()[0]
        group = group[["rho", w.value]].groupby(by=[w.value]).mean().reset_index()
        x = group[w.value]
        E = group["rho"]
        # ax.plot(
        #     [w.iloc[0],w.iloc[-1]],
        #     [E.iloc[0],E.iloc[-1]],
        #     # label=f"{name}",
        # )
        ax.scatter(x, E, label=f"{name}")

        ax.set_title(f"Rho - {w.value}")
        ax.set_xlabel(f"{w.value} [%]")
        ax.set_ylabel("rho [g/ml]")
    plt.legend()
    plt.show()